# 🚀 JetX Predictor - 5 Model Ensemble Orkestrasyon (v5.0 ULTIMATE)

Bu notebook, JetX projesinin **2 Modlu (Normal/Rolling)** sistemine uygun TÜM modelleri sırasıyla eğiten, Meta-Model ile birleştiren ve sonuçları Google Drive'a yedekleyen tam otomatize bir **Orkestrasyon Dosyasıdır**.

### 📋 Sistem Mimarisi:
1. **Progressive Transformer** (Monolitik Script)
   - *Mimari:* N-Beats + TCN + Transformer (Attention)
   - *Girdi:* 1000'lik zaman serisi
2. **Ultra Aggressive Model** (Monolitik Script)
   - *Mimari:* Derinleştirilmiş Hybrid NN
   - *Özellik:* Para kaybı odaklı 'Threshold Killer' loss
3. **CatBoost Ensemble**
   - *Yapı:* Multi-scale Gradient Boosting
4. **AutoGluon AutoML**
   - *Yapı:* 50+ model otomatik seçim
5. **TabNet Specialist**
   - *Yapı:* Attention-based tabular learning
6. **Meta-Model (Stacking)**
   - *Görevi:* Tüm modellerin kararlarını 0.85 ve 0.95 eşiklerine göre birleştirir.

### ⚡ Özellikler:
- ✅ **2 Modlu Yapı:** Normal (%85) ve Rolling (%95) için optimize edildi
- ✅ **Monolitik Scriptler:** Tek dosya eğitim (Dependency free)
- ✅ **Google Drive Entegrasyonu:** Otomatik yedekleme
- ✅ **Threshold Manager:** Merkezi eşik yönetimi

**Toplam Süre**: ~4-6 saat (GPU ile)

---

**🎯 KULLANIM:**
1. Bu notebook'u Google Colab'da açın
2. Runtime → Change runtime type → GPU (T4 önerilir)
3. Hücreleri sırasıyla çalıştırın (Cell → Run All)
4. `JetX_FULL_ARMY_v5.0.zip` dosyası otomatik indirilecektir

In [ ]:
# 🛠️ HÜCRE 1: ROBUST ORTAM KURULUMU
# Gerekli kütüphaneleri kur ve projeyi hazırla

import os
import sys
import subprocess
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 JetX Predictor v5.0 - Environment Setup")
print("="*80)
print(f"Başlangıç: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Google Drive mount
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive başarıyla bağlandı")
    
    # Backup dizini oluştur
    backup_dir = '/content/drive/MyDrive/JetX_Models_Backup_v5'
    os.makedirs(backup_dir, exist_ok=True)
    print(f"✅ Backup dizini hazır: {backup_dir}")
except:
    print("⚠️ Google Colab dışında çalıştırılıyor veya Drive bağlanamadı")

print("\n📦 GEREKLİ PAKETLER KURULUYOR... (Bu işlem birkaç dakika sürebilir)")

install_order = [
    # Temel Veri İşleme
    ['numpy', 'pandas', 'scikit-learn', 'joblib', 'scipy'],
    # Deep Learning Frameworks
    ['tensorflow', 'torch'],
    # Boosting & AutoML
    ['catboost', 'xgboost', 'lightgbm', 'autogluon', 'pytorch-tabnet'],
    # Optimizasyon & İstatistiksel Analiz
    ['optuna', 'shap', 'nolds', 'PyWavelets'],
    # Görselleştirme & Araçlar
    ['matplotlib', 'seaborn', 'tqdm', 'pyyaml']
]

for i, packages in enumerate(install_order, 1):
    print(f"\n📦 Grup {i}/{len(install_order)}: {', '.join(packages)}")
    for package in packages:
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', package], check=False)
            print(f"  ✅ {package}")
        except Exception as e:
            print(f"  ⚠️ {package} kurulum hatası: {e}")

print("\n📥 PROJE KURULUMU...")
PROJECT_ROOT = '/content/jetxpredictor'

if os.path.exists(PROJECT_ROOT):
    print(f"✅ Proje dizini mevcut: {PROJECT_ROOT}")
    os.chdir(PROJECT_ROOT)
    print("🔄 Güncel kodlar çekiliyor...")
    subprocess.run(['git', 'pull'], check=False)
else:
    print(f"📥 Proje GitHub'dan çekiliyor...")
    subprocess.run(['git', 'clone', 'https://github.com/onndd/jetxpredictor.git'], check=True)
    os.chdir(PROJECT_ROOT)
    print(f"✅ Proje klonlandı")

# Python path'e ekle
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("\n✅ ORTAM HAZIR!")

In [ ]:
# 🎛️ HÜCRE 2: KONTROL PANELİ

print("="*80)
print("🎛️ JETX TRAINING CONFIG (v5.0)")
print("="*80)

# 🚀 MODEL EĞİTİM BAYRAKLARI
# Hangi modellerin eğitileceğini buradan seçebilirsiniz
TRAIN_PROGRESSIVE = True    # Monolitik Progressive Transformer (En Önemli)
TRAIN_ULTRA = True          # Ultra Aggressive Model
TRAIN_CATBOOST = True       # CatBoost Ensemble
TRAIN_AUTOGLUON = True      # AutoGluon AutoML
TRAIN_TABNET = True         # TabNet Specialist
TRAIN_META_MODEL = True     # Stacking Meta-Model

# ⚡ HIZLI TEST MODU
# True: Epoch sayılarını düşürür (Test için)
# False: Tam eğitim yapar (Production için)
QUICK_TEST_MODE = False

# 📊 EĞİTİM PARAMETRELERİ
TRAINING_CONFIG = {
    'epochs': 5 if QUICK_TEST_MODE else 100,
    'catboost_iterations': 100 if QUICK_TEST_MODE else 1500,
    'autogluon_time': 60 if QUICK_TEST_MODE else 3600,
    'batch_size': 16 if QUICK_TEST_MODE else 32
}

MODELS_DIR = 'models'
os.makedirs(MODELS_DIR, exist_ok=True)

print(f"⚡ QUICK_TEST_MODE: {QUICK_TEST_MODE}")
print(f"🎯 HEDEFLER: Normal >%85, Rolling >%95")
print(f"📁 Modeller Klasörü: {MODELS_DIR}")

In [ ]:
# 📊 HÜCRE 3: VERİ HAZIRLIĞI VE KONTROLÜ
# Veritabanı yoksa oluşturur, varsa temizler ve hazırlar

import sqlite3
import pandas as pd
import numpy as np

print("📊 Veri kontrol ediliyor...")
db_path = 'jetx_data.db'

def create_synthetic_data(num_samples=3000):
    """Eğer veri yoksa eğitim için sentetik veri oluşturur"""
    print(f"⚠️ Veritabanı bulunamadı. {num_samples} adet sentetik veri oluşturuluyor...")
    # Log-normal dağılım (JetX benzeri)
    values = np.random.lognormal(0.5, 0.8, num_samples)
    values = np.clip(values, 1.0, 100.0)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS jetx_results (id INTEGER PRIMARY KEY AUTOINCREMENT, value REAL, timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)")
    
    for val in values:
        cursor.execute("INSERT INTO jetx_results (value) VALUES (?)", (val,))
    
    conn.commit()
    conn.close()
    print(f"✅ Sentetik veri oluşturuldu ve kaydedildi.")

if not os.path.exists(db_path):
    create_synthetic_data()
else:
    conn = sqlite3.connect(db_path)
    # Tablo var mı kontrol et
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='jetx_results'")
    if not cursor.fetchone():
        create_synthetic_data()
    else:
        df = pd.read_sql_query("SELECT value FROM jetx_results", conn)
        if len(df) < 100:
            create_synthetic_data()
        else:
            print(f"✅ {len(df)} adet gerçek veri bulundu.")
    conn.close()

In [ ]:
# 🧠 HÜCRE 4: PROGRESSIVE TRANSFORMER EĞİTİMİ (MONOLITHIC)
# Bu, sistemin ana modelidir. N-Beats, TCN ve Transformer katmanlarını içerir.

if TRAIN_PROGRESSIVE:
    print("\n" + "="*80)
    print("🧠 PROGRESSIVE TRANSFORMER EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        # Monolitik scripti çalıştır
        # Script kendi içinde her şeyi halleder (Feature eng, Model build, Train, Save)
        # Standalone çalıştığı için dışarıdan import hatası vermez
        subprocess.run([sys.executable, 'notebooks/jetx_PROGRESSIVE_TRAINING.py'], check=True)
        print("✅ Progressive Transformer eğitimi başarıyla tamamlandı.")
        
        # Kontrol
        if os.path.exists('models/jetx_progressive_final.h5'):
            print("💾 Model dosyası doğrulandı: models/jetx_progressive_final.h5")
            # Diğer scriptler için kopyala
            import shutil
            shutil.copy('models/jetx_progressive_final.h5', 'models/jetx_progressive_transformer.h5')
    except subprocess.CalledProcessError as e:
        print(f"❌ EĞİTİM HATASI (Progressive): Script hata kodu ile döndü: {e.returncode}")
    except Exception as e:
        print(f"❌ BEKLENMEYEN HATA: {e}")
else:
    print("⏭️ Progressive eğitimi atlandı.")

In [ ]:
# ⚡ HÜCRE 5: ULTRA AGGRESSIVE MODEL EĞİTİMİ (MONOLITHIC)
# Para kaybını minimize etmeye odaklanan özel model.

if TRAIN_ULTRA:
    print("\n" + "="*80)
    print("⚡ ULTRA AGGRESSIVE MODEL EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        subprocess.run([sys.executable, 'notebooks/jetx_model_training_ULTRA_AGGRESSIVE.py'], check=True)
        print("✅ Ultra Aggressive model eğitimi başarıyla tamamlandı.")
        
        if os.path.exists('models/jetx_ultra_model.h5'):
            print("💾 Model dosyası doğrulandı: models/jetx_ultra_model.h5")
    except subprocess.CalledProcessError as e:
        print(f"❌ EĞİTİM HATASI (Ultra): Script hata kodu ile döndü: {e.returncode}")
    except Exception as e:
        print(f"❌ BEKLENMEYEN HATA: {e}")
else:
    print("⏭️ Ultra Aggressive eğitimi atlandı.")

In [ ]:
# 🚀 HÜCRE 6: CATBOOST ENSEMBLE EĞİTİMİ
# Gradient Boosting gücü ile 10 farklı modelin birleşimi.

if TRAIN_CATBOOST:
    print("\n" + "="*80)
    print("🚀 CATBOOST ENSEMBLE EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        subprocess.run([sys.executable, 'notebooks/jetx_CATBOOST_TRAINING_MULTISCALE.py'], check=True)
        print("✅ CatBoost eğitimi başarıyla tamamlandı.")
    except subprocess.CalledProcessError as e:
        print(f"❌ EĞİTİM HATASI (CatBoost): {e}")
else:
    print("⏭️ CatBoost eğitimi atlandı.")

In [ ]:
# 🤖 HÜCRE 7: AUTOGLUON EĞİTİMİ
# AutoML ile en iyi modeli otomatik seçer.

if TRAIN_AUTOGLUON:
    print("\n" + "="*80)
    print("🤖 AUTOGLUON AUTOML EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        from utils.autogluon_predictor import AutoGluonPredictor
        from utils.database import DatabaseManager
        from category_definitions import FeatureEngineering
        
        print("📊 Veri hazırlanıyor...")
        db = DatabaseManager('jetx_data.db')
        data = db.get_all_results()
        
        # Veri yeterli mi?
        if len(data) > 500:
            # Feature extraction (Son 2000 veri için)
            X, y = [], []
            # Eğitim için geçmiş veriyi kullan
            limit = min(len(data), 3000)
            for i in range(500, limit):
                 hist = data[i-500:i]
                 target = data[i]
                 # Basit feature seti (hız için)
                 feats = FeatureEngineering.extract_all_features(hist)
                 X.append(list(feats.values()))
                 y.append(1 if target >= 1.5 else 0)
            
            X_df = pd.DataFrame(X)
            y_series = pd.Series(y)
            
            print(f"🚀 AutoGluon eğitiliyor (Süre limiti: {TRAINING_CONFIG['autogluon_time']}s)...")
            predictor = AutoGluonPredictor()
            predictor.train(X_df, y_series, time_limit=TRAINING_CONFIG['autogluon_time'])
            print("✅ AutoGluon eğitimi tamamlandı.")
        else:
            print("⚠️ Yetersiz veri, AutoGluon atlandı.")
        
    except Exception as e:
        print(f"❌ AutoGluon Hata: {e}")
else:
    print("⏭️ AutoGluon eğitimi atlandı.")

In [ ]:
# 🎯 HÜCRE 8: TABNET EĞİTİMİ

if TRAIN_TABNET:
    print("\n" + "="*80)
    print("🎯 TABNET SPECIALIST EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        from utils.tabnet_predictor import TabNetHighXPredictor
        from utils.database import DatabaseManager
        from category_definitions import FeatureEngineering
        import joblib
        from sklearn.preprocessing import StandardScaler
        
        print("📊 Veri hazırlanıyor...")
        db = DatabaseManager('jetx_data.db')
        data = db.get_all_results()
        
        if len(data) > 500:
            X, y = [], []
            limit = min(len(data), 3000)
            for i in range(500, limit):
                 hist = data[i-500:i]
                 target = data[i]
                 feats = FeatureEngineering.extract_all_features(hist)
                 X.append(list(feats.values()))
                 # Kategori tahmini (0, 1, 2, 3)
                 cat = TabNetHighXPredictor.categorize_value(target)
                 y.append(cat)
            
            X = np.array(X)
            y = np.array(y)
            
            # Scaling
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            joblib.dump(scaler, 'models/tabnet_scaler.pkl')
            
            print("🚀 TabNet eğitiliyor...")
            predictor = TabNetHighXPredictor()
            predictor.train(X_scaled, y, max_epochs=TRAINING_CONFIG['epochs'])
            predictor.save_model()
            print("✅ TabNet eğitimi tamamlandı.")
        else:
            print("⚠️ Yetersiz veri, TabNet atlandı.")
            
    except Exception as e:
        print(f"❌ TabNet Hata: {e}")
else:
    print("⏭️ TabNet eğitimi atlandı.")

In [ ]:
# 🏆 HÜCRE 9: META-MODEL EĞİTİMİ (STACKING)
# Tüm modelleri birleştirip nihai kararı verir.

if TRAIN_META_MODEL:
    print("\n" + "="*80)
    print("🏆 META-MODEL EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        # TRAIN_META_MODEL.py dosyasını çalıştır
        subprocess.run([sys.executable, 'notebooks/TRAIN_META_MODEL.py'], check=True)
        print("✅ Meta-Model eğitimi başarıyla tamamlandı.")
    except subprocess.CalledProcessError as e:
        print(f"❌ EĞİTİM HATASI (Meta-Model): {e}")
    except Exception as e:
        print(f"❌ BEKLENMEYEN HATA: {e}")
else:
    print("⏭️ Meta-Model eğitimi atlandı.")

In [ ]:
# 📦 HÜCRE 10: PAKETLEME VE İNDİRME

print("\n" + "="*80)
print("📦 PAKETLEME VE İNDİRME")
print("="*80)

import shutil
import zipfile

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'JetX_FULL_ARMY_v5.0_{timestamp}.zip'

# Dosyaları topla
files_to_zip = [
    'models/jetx_progressive_transformer.h5',
    'models/jetx_progressive_final.h5', # Yedek
    'models/scaler_progressive.pkl',
    'models/jetx_ultra_model.h5',
    'models/scaler_ultra.pkl',
    'models/meta_model.json',
    'models/meta_model_info.json',
    'models/autogluon_model', # Klasör
    'models/autogluon_scaler.pkl',
    'models/tabnet_high_x.pkl',
    'models/tabnet_scaler.pkl',
    'models/catboost_multiscale' # Klasör
]

print("Dosyalar sıkıştırılıyor...")
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for item in files_to_zip:
        if os.path.exists(item):
            if os.path.isdir(item):
                for root, dirs, files in os.walk(item):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, os.path.join(item, '..'))
                        zipf.write(file_path, arcname)
            else:
                zipf.write(item, os.path.basename(item))
            print(f"  Adding: {item}")
        else:
            print(f"  ⚠️ Bulunamadı: {item}")

# Google Drive'a kopyala
try:
    if os.path.exists('/content/drive'):
        backup_path = f'/content/drive/MyDrive/JetX_Models_Backup_v5/{zip_filename}'
        shutil.copy(zip_filename, backup_path)
        print(f"\n✅ Google Drive'a yedeklendi: {backup_path}")
except Exception as e:
    print(f"⚠️ Drive yedekleme hatası: {e}")

# İndirme
try:
    from google.colab import files
    files.download(zip_filename)
    print("\n⬇️ İndirme başlatıldı...")
except:
    print(f"\n⚠️ Otomatik indirme başarısız. Dosya yolu: {os.path.abspath(zip_filename)}")

print("\n🎉 İŞLEM TAMAMLANDI!")